In [2]:
import pandas as pd
from os import getcwd
from sqlalchemy.engine import URL
from sqlalchemy import create_engine
# from UliPlot.XLSX import auto_adjust_xlsx_column_width

In [26]:
strain_query = '''
SELECT  [StrainGauge].[gid]
      ,[gpn]
      ,[ver] as 'PCBver'
      ,[toolno]
      ,[station]
      ,[Position]
      ,[TestRecord].judge
      ,[TestRecord].stress
      ,[TestRecord].WQ211
      ,[memo]
      ,[org]
      ,[createdata]
  FROM [SMTvip].[dbo].[StrainGauge]

  Left JOIN [SMTvip].[dbo].[TestRecord] 
  on [StrainGauge].[gid] = [TestRecord].[gid]
  where gpn in ('105-04714-00','105-04714-01')
  ORDER by profdate DESC
'''

connection_string = "DRIVER={SQL Server};SERVER=T1-PE-SUPPORT;DATABASE=PR_WEB2;UID=pimes2;PWD=LhZEv9JtGsZNX"
connection_url = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})
engine = create_engine(connection_url)
raw = pd.read_sql(strain_query, engine)

In [27]:
raw.sample(5)

,gid,gpn,PCBver,toolno,station,Position,judge,stress,WQ211,memo,org,createdata
23,15369,105-04714-01,V2,3,PRNP,I101,OK,22.87,300,PRNP(治具編號:3.4),T1,2023-08-10 11:00:00
11,15405,105-04714-01,V2,無,Other,I502,OK,225.00,250,SMT組鐵蓋,T1,2023-08-14 18:08:00
43,15139,105-04714-01,V1,無,折板,I505,OK,259.27,300,Router2.0使用105-04714-00V2線板驗證,T1,2023-05-24 14:56:00
28,15369,105-04714-01,V2,3,PRNP,I509,OK,59.26,300,PRNP(治具編號:3.4),T1,2023-08-10 11:00:00
20,15364,105-04714-01,V2,5,ATE,I509,OK,62.82,300,ATE2.0(治具編號:5.6),T1,2023-08-07 18:13:00


In [30]:
def pivot_raw(raw):
    return(raw
    .assign(judge = raw.judge.map({'OK':1, 'NG':0}).astype('int8'))
    .sort_values('createdata', ascending=False)
    .drop_duplicates(subset=['PCBver','station','Position','memo','judge']) #The reason subset consider 'judge' is same position may have 2 strain gauge and each of gauge may have different result
    # .query("Position == 'I401'")
    .groupby(['station','memo','PCBver','createdata','Position'])
    ['judge']
    .min() #If one of results in the same position is 0, take 0 as output. 
    .unstack('Position')
    .sort_index(level=['station','createdata'],ascending=False)
    .style.applymap(lambda val : "background-color: red" if val == 0 else "")
    )
pivot_raw(raw)

In [ ]:
with pd.ExcelWriter('StrainResult.xlsx', engine='openpyxl') as writer:
    pivot_raw(raw).to_excel(writer, sheet_name='pivot')
    raw.to_excel(writer, sheet_name='raw', index=False)